# Question answering using embeddings-based search

In [3]:
# imports
import ast  # for converting embeddings saved as strings back to arrays
import openai  # for calling the OpenAI API
import pandas as pd  # for storing text and embeddings data
import tiktoken  # for counting tokens
from scipy import spatial  # for calculating vector similarities for search


# models
EMBEDDING_MODEL = "text-embedding-ada-002"
GPT_MODEL = "gpt-3.5-turbo"

from keys import *
openai.api_key = OPENAI_API_KEY

In [6]:
embeddings_path = "ans_generator/data/ir_book_embeddings.csv"

df = pd.read_csv(embeddings_path)

In [7]:
# convert embeddings from CSV str type back to list type
df['embedding'] = df['embedding'].apply(ast.literal_eval)

In [8]:
# the dataframe has two columns: "text" and "embedding"
df

,text,embedding
0,ChengXiang Zhai Sean Massung Text Data Managem...,"[-0.017203252762556076, 0.021963899955153465, ..."
1,This has led to an increasing demand for power...,"[-0.016908559948205948, 0.010532873682677746, ..."
2,Unlike data generated by a computer system or ...,"[-0.018540382385253906, 0.014797425828874111, ..."
3,As such text data are especially valuable for ...,"[-0.020149044692516327, 0.02085673063993454, 0..."
4,In contrast to structured data which conform t...,"[-0.02140810526907444, 0.02335185930132866, 0...."
...,...,...
8045,Please note that this α is different from αd i...,"[0.0068044946528971195, 0.035624921321868896, ..."
8046,Of course the next question is how to estimate...,"[0.007543814368546009, 0.032577402889728546, -..."
8047,where F d1 dk is the set of feedback document...,"[-0.016099639236927032, 0.021384460851550102, ..."
8048,Now given λ the feedback documents F and the ...,"[-0.014733465388417244, 0.021942878141999245, ..."


In [10]:
# search function
def strings_ranked_by_relatedness(
    query: str,
    df: pd.DataFrame,
    relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y),
    top_n: int = 100
) -> tuple[list[str], list[float]]:
    """Returns a list of strings and relatednesses, sorted from most related to least."""
    query_embedding_response = openai.Embedding.create(
        model=EMBEDDING_MODEL,
        input=query,
    )
    query_embedding = query_embedding_response["data"][0]["embedding"]
    strings_and_relatednesses = [
        (row["text"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()
    ]
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    strings, relatednesses = zip(*strings_and_relatednesses)
    return strings[:top_n], relatednesses[:top_n]


In [11]:
# examples
strings, relatednesses = strings_ranked_by_relatedness("Web Crawling", df, top_n=5)
for string, relatedness in zip(strings, relatednesses):
    print(f"{relatedness=:.3f}")
    display(string)

relatedness=0.885


'10. 1 Web Crawling The crawler is also called a spider or a software robot that crawls traverses parses and downloads pages on the web.  Building a toy crawler is relatively easy because you just need to start with a set of seed pages fetch pages from the web and parse these pages’ new links.  We then add them to a queue and then explore those page’s links in a breadthfirst search until we are satisfied.  Building a real crawler is quite tricky and there are some complicated issues that we inevitably deal with.'

relatedness=0.875


'We’ve already described all indexing steps except crawling in detail in Chapter 8.  After our crawling discussion we move onto the particular challenges of web indexing.  Then we discuss how we can take advantage of links between pages in link analysis.  The last technique we discuss is learning to rank which is a way to combine many different features for ranking.  10. 1 Web Crawling The crawler is also called a spider or a software robot that crawls traverses parses and downloads pages on the web.  Building a toy crawler is relatively easy because you just need to start with a set of seed pages fetch pages from the web and parse these pages’ new links.'

relatedness=0.874


'1 Web Crawling The crawler is also called a spider or a software robot that crawls traverses parses and downloads pages on the web.  Building a toy crawler is relatively easy because you just need to start with a set of seed pages fetch pages from the web and parse these pages’ new links.  We then add them to a queue and then explore those page’s links in a breadthfirst search until we are satisfied.  Building a real crawler is quite tricky and there are some complicated issues that we inevitably deal with.  One issue is robustness What if the server doesn’t respond or returns unparseable garbage What if there’s a trap that generates dynamically generated pages that attract your crawler to keep crawling the same site in circles Yet another issue is that we don’t want to overload one particular server with too many crawling requests.  Those may cause the site to experience a denial of service some sites will also block IP addresses that they believe to be crawling them or creating too 

relatedness=0.873


'In the next section we will discuss crawling.  We’ve already described all indexing steps except crawling in detail in Chapter 8.  After our crawling discussion we move onto the particular challenges of web indexing.  Then we discuss how we can take advantage of links between pages in link analysis.  The last technique we discuss is learning to rank which is a way to combine many different features for ranking.  10. 1 Web Crawling The crawler is also called a spider or a software robot that crawls traverses parses and downloads pages on the web.  Building a toy crawler is relatively easy because you just need to start with a set of seed pages fetch pages from the web and parse these pages’ new links.'

relatedness=0.873


'One interesting variation is called focused crawling.  Here we’re going to crawl some pages about a particular topic eg all pages about automobiles.  This is typically going to start with a query that you use to get some results.  Then you gradually crawl more.  An even more extreme version of focused crawling is for example downloading and indexing all forum posts on a particular forum.  In this case we might have a URL such as httpwww. textdatabookforum. comboardsid3 which refers to the third post on the forum.'

In [25]:
def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Return the number of tokens in a string."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))


def query_message(
    query: str,
    df: pd.DataFrame,
    model: str,
    token_budget: int
) -> str:
    """Return a message for GPT, with relevant source texts pulled from a dataframe."""
    strings, relatednesses = strings_ranked_by_relatedness(query, df)
    introduction = "Suppose you are a teaching assistant for the course Advanced Information Retrieval and a student has posed the following question.\n"
    question = f"\n\nQuestion: {query}\n\n"
    end = 'How will you answer the question? \n Here are some snippets from the course textbook which may be useful.\n\n"'
    book_info = ""
    
    preface = introduction + question + end
    for string in strings:
        next_article = f'\n{string}\n'
        if (
            num_tokens(preface + book_info + next_article, model=model)
            > token_budget
        ):
            break
        else:
            book_info += next_article
    return preface + book_info

def api_call(message, model: str = GPT_MODEL):
    messages = [
        {"role": "user", "content": message},
    ]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0
    )
    response_message = response["choices"][0]["message"]["content"]
    return response_message

def ask(
    query: str,
    df: pd.DataFrame = df,
    model: str = GPT_MODEL,
    token_budget: int = 4096 - 500,
    print_message: bool = False,
) -> str:
    """Answers a query using GPT and a dataframe of relevant texts and embeddings."""
    message = query_message(query, df, model=model, token_budget=token_budget)
    if print_message:
        print(message)
    
    reply = api_call(message, model)
    return reply

In [27]:
query = 'When modeling queries, how is multi-bernoulli different from multinomial?'

api_call(query)
# ask('When modeling queries, how is multi-bernoulli different from multinomial?')

'Multi-Bernoulli and Multinomial are two different models used in probability theory and statistics to model queries. \n\nMulti-Bernoulli is a model used to represent a sequence of binary events, where each event can have one of two outcomes (success or failure). It is used to model queries where the data is binary, such as in object tracking or sensor fusion. In Multi-Bernoulli, the probability of success or failure is assumed to be constant across all events.\n\nOn the other hand, Multinomial is a model used to represent a sequence of events where each event can have one of several outcomes. It is used to model queries where the data is categorical, such as in natural language processing or image classification. In Multinomial, the probability of each outcome is assumed to be constant across all events.\n\nIn summary, Multi-Bernoulli is used for binary data, while Multinomial is used for categorical data.'

In [30]:
def compare_responses(query):
    emb_reply = ask(query)
    base_reply = api_call(query)

    print(f'Query:{query}\n\n')
    print("Embedding Method response\n")
    print(emb_reply+"\n\n")
    print('Base GPT response\n')
    print(base_reply)

In [31]:
query = 'How do you define a reference language model?'

compare_response(query)

Query:How do you define a reference language model?


Embedding Method response

A reference language model is a probability distribution over word sequences that is used to estimate the probability of an unseen word in a corpus. If a word is not observed in the corpus, its probability is assumed to be governed by the reference language model. In the case of retrieval, a natural choice for the reference language model would be the collection language model. The collection language model is used to adjust the probability of the maximum likelihood estimate of seen terms. The general English language model can be used as a background language model to filter out common words and obtain a probability ratio for each word. Language models are useful for quantifying the uncertainties associated with the use of natural language and can help answer many interesting questions related to text analysis and information retrieval. Neural language models can be represented as a neural network and can

In [32]:
query = 'How can we use language models for part of speech tagging?'

compare_response(query)

Query:How can we use language models for part of speech tagging?


Embedding Method response

To use language models for part of speech tagging, we can assign a part of speech tag to each word based on its surrounding words. This allows us to count the most frequent nouns or determine what kind of nouns are associated with what kind of verbs. We can also consider n-grams of part of speech tags and mix n-grams of words and part of speech tags to create hybrid features that can be useful for sentiment analysis. Additionally, we can use language models to perform semantic analysis of word relations by finding what words are semantically associated with a given word. However, it is important to note that while part of speech tagging is a relatively easy task, full structure parsing and semantic analysis remain difficult and are only successful for some aspects of analysis. Overall, using language models for part of speech tagging can enrich the representation of text data and enable deeper

In [33]:
query = 'What happens to the Dirichlet smoothing when the document langeth goes to infinity'

compare_response(query)

Query:What happens to the Dirichlet smoothing when the document langeth goes to infinity


Embedding Method response

When the document length goes to infinity, the effect on Dirichlet smoothing is that the coefficient αd tends to zero. This is because αd penalizes long documents, and as the document length increases, the number of observed words also increases, making it less necessary to do smoothing. Therefore, the Dirichlet smoothing becomes less effective as the document length increases towards infinity. However, it is important to note that this effect is not unique to Dirichlet smoothing and is a general property of smoothing methods that penalize long documents.


Base GPT response

When the document length goes to infinity, the effect of Dirichlet smoothing decreases. This is because as the document length increases, the probability estimates become more accurate and the need for smoothing decreases. In other words, as the amount of data increases, the impact of the prior dis